In [1]:
# =============================================================================
# Cell 1: Reproducibility
# =============================================================================
SEED = 5
import os, random
import numpy as np
import tensorflow as tf

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-10-22 17:07:26.188433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761152846.369641      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761152846.426300      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# =============================================================================
# Cell 2: Core Imports and Constants
# =============================================================================
import json, math, os, gc
import cv2
from PIL import Image
import pickle
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (
    EfficientNetV2M, EfficientNetB5, InceptionV3, Xception, DenseNet169,
    DenseNet121, InceptionResNetV2, ResNet50
)
# Machine Learning
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.ensemble import VotingClassifier
import catboost as cb
import scipy
from scipy.special import softmax
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
# XAI Libraries
import shap
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
import seaborn as sns
# Jupyter
%matplotlib inline

# Constants
IMG_SIZE = 224
BATCH_SIZE = 24
EPOCHS = 50
PATIENCE = 7

# GPU Memory Configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")

# Enable XLA compilation
tf.config.optimizer.set_jit(True)
print("✅ XLA compilation enabled")

# Kaggle input paths
APTOS_DIR = "/kaggle/input/aptos2019-blindness-detection"
PROCESSED_DATA_PATH = "/kaggle/input/please/preprocessed_aptos_data.npz"
TEST_CSV_PATH = "/kaggle/input/please/test_data.csv"
TRAIN_CSV_PATH = "/kaggle/input/please/train_data.csv"

# Class labels
CLASS_LABELS = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']

print("✅ All imports completed successfully")

✅ GPU memory growth enabled for 1 GPU(s)
✅ XLA compilation enabled
✅ All imports completed successfully


In [3]:
# =============================================================================
# Cell 3: Load Preprocessed Data - FIXED FOR EFFICIENTNET
# =============================================================================
print("="*80)
print("LOADING DATA FOR EFFICIENTNET TRAINING")
print("="*80)

# Load preprocessed data
data = np.load(PROCESSED_DATA_PATH)
print("Available keys:", list(data.keys()))

# Load data
x_train_full = data['X_train']
y_train_full = data['y_train']
x_test = data['X_test']  
y_test = data['y_test']

# Load CSV files
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

print(f"\n📊 Original data shapes:")
print(f"   Training data: {x_train_full.shape}")
print(f"   Training labels: {y_train_full.shape}")
print(f"   Test data: {x_test.shape}")
print(f"   Test labels: {y_test.shape}")

# CRITICAL: Check and convert labels to one-hot encoding
if len(y_train_full.shape) == 1 or (len(y_train_full.shape) == 2 and y_train_full.shape[1] == 1):
    print("\n🔄 Converting labels to one-hot encoding...")
    from tensorflow.keras.utils import to_categorical
    y_train_full_onehot = to_categorical(y_train_full, num_classes=5)
    y_test_onehot = to_categorical(y_test, num_classes=5)
else:
    y_train_full_onehot = y_train_full
    y_test_onehot = y_test

# Store original integer labels for evaluation
if len(y_train_full.shape) > 1:
    y_train_original = np.argmax(y_train_full, axis=1)
    y_test_original = np.argmax(y_test, axis=1)
else:
    y_train_original = y_train_full.copy()
    y_test_original = y_test.copy()

print(f"\n✅ Label encoding completed:")
print(f"   One-hot shape: {y_train_full_onehot.shape}")
print(f"   Original integer labels saved for evaluation")

# CRITICAL FIX: Keep images in [0-255] range for EfficientNet
print(f"\n🔧 EFFICIENTNET PREPROCESSING:")
print(f"   Current data range: [{x_train_full.min():.2f}, {x_train_full.max():.2f}]")

if x_train_full.max() <= 1.0:
    # Data is normalized [0-1], convert back to [0-255]
    print("   ⚠️  Data is normalized. Converting to [0-255] for EfficientNet...")
    x_train_full = (x_train_full * 255.0).astype('float32')
    x_test = (x_test * 255.0).astype('float32')
    print(f"   ✅ Converted to range: [{x_train_full.min():.2f}, {x_train_full.max():.2f}]")
elif x_train_full.max() > 1.0:
    print("   ✅ Data already in [0-255] range")
    x_train_full = x_train_full.astype('float32')
    x_test = x_test.astype('float32')

# Verify class distribution
print(f"\n📊 Class distribution:")
print(f"   Training: {np.bincount(y_train_original)}")
print(f"   Test: {np.bincount(y_test_original)}")

print("\n✅ Data loaded and prepared for EfficientNetB5 training")
print("="*80)


LOADING DATA FOR EFFICIENTNET TRAINING
Available keys: ['X_train', 'y_train', 'X_test', 'y_test']

📊 Original data shapes:
   Training data: (7220, 224, 224, 3)
   Training labels: (7220, 5)
   Test data: (733, 224, 224, 3)
   Test labels: (733,)


AxisError: axis 1 is out of bounds for array of dimension 1

In [13]:
# =============================================================================
# Grad-CAM Visualization Script for Diabetic Retinopathy Models
# Generates Grad-CAM heatmaps for 50 images (10 per class) across 4 models
# Now saves original and overlay images separately
# =============================================================================

print("="*80)
print("GRAD-CAM VISUALIZATION SCRIPT")
print("="*80)

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
import os

# =============================================================================
# Cell 1: Grad-CAM Implementation
# =============================================================================

def get_last_conv_layer_name(model):
    """
    Automatically find the last convolutional layer in the model
    """
    # First check the main model layers
    for layer in reversed(model.layers):
        # Check if it's a convolutional layer by class name
        if 'Conv' in layer.__class__.__name__:
            return layer.name
    
    # If no conv layer found in main model, check inside base model (first layer)
    if hasattr(model.layers[0], 'layers'):
        for layer in reversed(model.layers[0].layers):
            if 'Conv' in layer.__class__.__name__:
                return layer.name
    
    raise ValueError("Could not find convolutional layer. Model might have unexpected architecture.")


def make_gradcam_heatmap_simple(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Simplified Grad-CAM that works with Sequential models containing nested base models
    """
    # Get base model
    base_model = model.layers[0]
    
    # Get the target convolutional layer
    target_layer = base_model.get_layer(last_conv_layer_name)
    
    # Create a sub-model from input to target conv layer
    submodel = tf.keras.Model(inputs=base_model.input, outputs=target_layer.output)
    
    # Use GradientTape to compute gradients
    with tf.GradientTape() as tape:
        # Get conv layer activations
        conv_output = submodel(img_array, training=False)
        tape.watch(conv_output)
        
        # Continue forward pass through remaining layers
        x = conv_output
        
        # Get all layers after target layer in base model
        start_applying = False
        for layer in base_model.layers:
            if start_applying:
                x = layer(x, training=False)
            if layer.name == last_conv_layer_name:
                start_applying = True
        
        # Apply remaining model layers (pooling, dense, etc.)
        for layer in model.layers[1:]:
            x = layer(x, training=False)
        
        preds = x
        
        # Get the predicted class
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        
        # Get the score for target class
        target_class_output = preds[:, pred_index]
    
    # Compute gradients
    grads = tape.gradient(target_class_output, conv_output)
    
    # Global average pooling on gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight channels by gradient importance and sum
    conv_output = conv_output[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)
    
    # Apply ReLU and normalize
    heatmap = tf.nn.relu(heatmap)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-10)
    
    return heatmap.numpy()


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Wrapper that calls the simplified version
    """
    return make_gradcam_heatmap_simple(img_array, model, last_conv_layer_name, pred_index)


def overlay_gradcam(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    """
    Overlay Grad-CAM heatmap on original image
    
    Args:
        img: Original image array (224, 224, 3) in range [0, 1]
        heatmap: Grad-CAM heatmap
        alpha: Transparency of heatmap overlay
        colormap: OpenCV colormap to use
    
    Returns:
        superimposed_img: Image with Grad-CAM overlay
    """
    # Resize heatmap to match image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # Convert image to uint8
    img_uint8 = np.uint8(255 * img)
    
    # Superimpose the heatmap on original image
    superimposed_img = heatmap * alpha + img_uint8
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')
    
    return superimposed_img


# =============================================================================
# Cell 2: Select 50 Images (10 per class)
# =============================================================================

def select_images_per_class(x_data, y_data, num_per_class=10, random_seed=42):
    """
    Select specified number of images per class with fixed random seed
    
    Args:
        x_data: Image data
        y_data: Labels (one-hot encoded or integer)
        num_per_class: Number of images to select per class
        random_seed: Random seed for reproducibility
    
    Returns:
        selected_indices: Indices of selected images
        selected_images: Selected image arrays
        selected_labels: Selected labels
    """
    # Set random seed for reproducibility
    np.random.seed(random_seed)
    
    # Convert one-hot to integer labels if needed
    if len(y_data.shape) > 1:
        labels = np.argmax(y_data, axis=1)
    else:
        labels = y_data
    
    selected_indices = []
    
    # Select images for each class
    for class_idx in range(5):
        # Get indices of all images for this class
        class_indices = np.where(labels == class_idx)[0]
        
        # Randomly select num_per_class images
        if len(class_indices) >= num_per_class:
            selected = np.random.choice(class_indices, num_per_class, replace=False)
        else:
            # If not enough images, select all available
            selected = class_indices
            print(f"Warning: Only {len(class_indices)} images available for class {class_idx}")
        
        selected_indices.extend(selected)
    
    selected_indices = np.array(selected_indices)
    selected_images = x_data[selected_indices]
    selected_labels = labels[selected_indices]
    
    return selected_indices, selected_images, selected_labels


print("\n📊 Selecting 50 images (10 per class) with fixed random seed...")
selected_indices, selected_images, selected_labels = select_images_per_class(
    x_test, y_test, num_per_class=10, random_seed=42
)

print(f"✅ Selected {len(selected_indices)} images")
print(f"   Class distribution: {np.bincount(selected_labels)}")


# =============================================================================
# Cell 3: Load Models and Generate Grad-CAM
# =============================================================================

# Model paths - UPDATE THESE PATHS BASED ON YOUR SAVED MODELS
MODEL_PATHS = {
    'EfficientNetV2M': '/kaggle/working/effnetv2m_final.h5',
    'DenseNet169': '/kaggle/working/densenet169_best.h5',
    'InceptionV3': '/kaggle/working/inceptionv3_final.h5',
    'EfficientNetB5': '/kaggle/working/effnetb5_final.h5'
}

# Check which models are available
available_models = {}
for model_name, model_path in MODEL_PATHS.items():
    if os.path.exists(model_path):
        available_models[model_name] = model_path
        print(f"✅ Found: {model_name}")
    else:
        print(f"⚠️  Not found: {model_name} at {model_path}")

if len(available_models) == 0:
    print("\n❌ No models found! Please update MODEL_PATHS with correct paths.")
    print("   Current directory contents:")
    print("  ", os.listdir('.'))
else:
    print(f"\n✅ Found {len(available_models)} model(s) to process")


# =============================================================================
# Cell 4: Generate Grad-CAM for All Models
# =============================================================================

def generate_gradcam_for_model(model_name, model_path, images, labels):
    """
    Generate Grad-CAM visualizations for a specific model
    """
    print(f"\n{'='*80}")
    print(f"Processing: {model_name}")
    print(f"{'='*80}")
    
    # Load model
    print(f"Loading model from {model_path}...")
    model = load_model(model_path)
    
    # Find last convolutional layer
    try:
        last_conv_layer = get_last_conv_layer_name(model)
        print(f"✅ Using convolutional layer: {last_conv_layer}")
    except ValueError as e:
        print(f"❌ Error: {e}")
        return
    
    # Create output directories
    output_dir = f'gradcam_{model_name.lower()}'
    original_dir = f'{output_dir}/original'
    overlay_dir = f'{output_dir}/overlay'
    combined_dir = f'{output_dir}/combined'
    
    os.makedirs(original_dir, exist_ok=True)
    os.makedirs(overlay_dir, exist_ok=True)
    os.makedirs(combined_dir, exist_ok=True)
    
    # Generate Grad-CAM for each image
    print(f"\n🔥 Generating Grad-CAM visualizations...")
    
    for idx, (img, true_label) in enumerate(zip(images, labels)):
        # Prepare image for model
        img_array = np.expand_dims(img, axis=0)
        
        # Get prediction
        preds = model.predict(img_array, verbose=0)
        pred_label = np.argmax(preds[0])
        confidence = preds[0][pred_label]
        
        # Generate Grad-CAM heatmap
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer, pred_index=pred_label)
        
        # Create overlay
        gradcam_img = overlay_gradcam(img, heatmap, alpha=0.4)
        
        # Base filename
        base_filename = f'{idx+1:02d}_class{true_label}_pred{pred_label}'
        
        # Save original image separately
        fig_orig = plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.savefig(f'{original_dir}/{base_filename}_original.png', 
                   dpi=150, bbox_inches='tight', pad_inches=0)
        plt.close(fig_orig)
        
        # Save overlay image separately
        fig_overlay = plt.figure(figsize=(5, 5))
        plt.imshow(gradcam_img)
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.savefig(f'{overlay_dir}/{base_filename}_overlay.png', 
                   dpi=150, bbox_inches='tight', pad_inches=0)
        plt.close(fig_overlay)
        
        # Create and save combined visualization
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Original image
        axes[0].imshow(img)
        axes[0].set_title(f'Original Image\nTrue: {CLASS_LABELS[true_label]}', fontsize=10)
        axes[0].axis('off')
        
        # Heatmap only
        axes[1].imshow(heatmap, cmap='jet')
        axes[1].set_title('Grad-CAM Heatmap', fontsize=10)
        axes[1].axis('off')
        
        # Overlay
        axes[2].imshow(gradcam_img)
        axes[2].set_title(f'Grad-CAM Overlay\nPred: {CLASS_LABELS[pred_label]} ({confidence:.2%})', fontsize=10)
        axes[2].axis('off')
        
        # Add main title
        correct = "✓" if true_label == pred_label else "✗"
        fig.suptitle(f'{model_name} - Image {idx+1}/50 {correct}', 
                     fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        
        # Save combined figure
        plt.savefig(f'{combined_dir}/{base_filename}_combined.png', 
                   dpi=150, bbox_inches='tight')
        plt.close(fig)
        
        if (idx + 1) % 10 == 0:
            print(f"   Processed {idx+1}/50 images...")
    
    print(f"✅ All visualizations saved to '{output_dir}/' directory")
    print(f"   - Original images: {original_dir}/")
    print(f"   - Overlay images: {overlay_dir}/")
    print(f"   - Combined views: {combined_dir}/")
    
    # Clean up
    del model
    tf.keras.backend.clear_session()


# Process all available models
for model_name, model_path in available_models.items():
    generate_gradcam_for_model(model_name, model_path, selected_images, selected_labels)


# =============================================================================
# Cell 5: Create Summary Visualization
# =============================================================================

print("\n" + "="*80)
print("CREATING SUMMARY COMPARISON")
print("="*80)

def create_summary_comparison(models_dict, images, labels, num_samples=5):
    """
    Create a summary comparison showing Grad-CAM from all models side-by-side
    """
    print(f"\nCreating summary comparison for {num_samples} sample images...")
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Randomly select sample images (one from each class)
    sample_indices = []
    for class_idx in range(min(5, num_samples)):
        class_mask = labels == class_idx
        class_imgs = np.where(class_mask)[0]
        if len(class_imgs) > 0:
            sample_indices.append(np.random.choice(class_imgs))
    
    # Load all models
    loaded_models = {}
    conv_layers = {}
    
    for model_name, model_path in models_dict.items():
        loaded_models[model_name] = load_model(model_path)
        conv_layers[model_name] = get_last_conv_layer_name(loaded_models[model_name])
    
    # Create comparison for each sample
    os.makedirs('gradcam_comparison', exist_ok=True)
    
    for sample_idx, img_idx in enumerate(sample_indices):
        img = images[img_idx]
        true_label = labels[img_idx]
        
        num_models = len(loaded_models)
        fig, axes = plt.subplots(1, num_models + 1, figsize=(5 * (num_models + 1), 5))
        
        # Original image
        axes[0].imshow(img)
        axes[0].set_title(f'Original\nTrue: {CLASS_LABELS[true_label]}', 
                         fontsize=11, fontweight='bold')
        axes[0].axis('off')
        
        # Grad-CAM from each model
        img_array = np.expand_dims(img, axis=0)
        
        for idx, (model_name, model) in enumerate(loaded_models.items()):
            # Get prediction
            preds = model.predict(img_array, verbose=0)
            pred_label = np.argmax(preds[0])
            confidence = preds[0][pred_label]
            
            # Generate Grad-CAM
            heatmap = make_gradcam_heatmap(img_array, model, conv_layers[model_name], pred_index=pred_label)
            gradcam_img = overlay_gradcam(img, heatmap, alpha=0.4)
            
            # Plot
            axes[idx + 1].imshow(gradcam_img)
            axes[idx + 1].set_title(f'{model_name}\n{CLASS_LABELS[pred_label]} ({confidence:.1%})', 
                                   fontsize=11, fontweight='bold')
            axes[idx + 1].axis('off')
        
        fig.suptitle(f'Model Comparison - Sample {sample_idx + 1} (Class: {CLASS_LABELS[true_label]})', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        # Save
        plt.savefig(f'gradcam_comparison/comparison_sample_{sample_idx + 1}.png', 
                   dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"✅ Summary comparisons saved to 'gradcam_comparison/' directory")
    
    # Clean up
    for model in loaded_models.values():
        del model
    tf.keras.backend.clear_session()


# Create summary comparison if multiple models available
if len(available_models) > 1:
    create_summary_comparison(available_models, selected_images, selected_labels, num_samples=5)

print("\n" + "="*80)
print("✅ GRAD-CAM VISUALIZATION COMPLETED!")
print("="*80)
print(f"\n📁 Output directories created:")
for model_name in available_models.keys():
    model_dir = f"gradcam_{model_name.lower()}"
    print(f"   - {model_dir}/original/  (50 original images)")
    print(f"   - {model_dir}/overlay/   (50 overlay images)")
    print(f"   - {model_dir}/combined/  (50 combined visualizations)")
if len(available_models) > 1:
    print(f"   - gradcam_comparison/  (5 comparison images)")
print("\n🎉 All Grad-CAM visualizations generated successfully!")

GRAD-CAM VISUALIZATION SCRIPT

📊 Selecting 50 images (10 per class) with fixed random seed...
✅ Selected 50 images
   Class distribution: [10 10 10 10 10]
✅ Found: EfficientNetV2M
⚠️  Not found: DenseNet169 at /kaggle/working/densenet169_best.h5
⚠️  Not found: InceptionV3 at /kaggle/working/inceptionv3_final.h5
⚠️  Not found: EfficientNetB5 at /kaggle/working/effnetb5_final.h5

✅ Found 1 model(s) to process

Processing: EfficientNetV2M
Loading model from /kaggle/working/effnetv2m_final.h5...
✅ Using convolutional layer: top_conv

🔥 Generating Grad-CAM visualizations...
   Processed 10/50 images...
   Processed 20/50 images...
   Processed 30/50 images...
   Processed 40/50 images...
   Processed 50/50 images...
✅ All visualizations saved to 'gradcam_efficientnetv2m/' directory
   - Original images: gradcam_efficientnetv2m/original/
   - Overlay images: gradcam_efficientnetv2m/overlay/
   - Combined views: gradcam_efficientnetv2m/combined/

CREATING SUMMARY COMPARISON

✅ GRAD-CAM VISUAL